In [1]:
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import zoom

In [2]:
os.getcwd()

'/home/nick/classwork_21_22/COMP755/COMP755FinalProj/Processing'

In [3]:
celeba256_f = sorted(glob('../../celeba_256_masked_png/*'))
print(len(celeba256_f))
print(celeba256_f[:5])


15028
['../../celeba_256_masked_png/000001.jpg_256_masked.png', '../../celeba_256_masked_png/000002.jpg_256_masked.png', '../../celeba_256_masked_png/000003.jpg_256_masked.png', '../../celeba_256_masked_png/000004.jpg_256_masked.png', '../../celeba_256_masked_png/000006.jpg_256_masked.png']


In [4]:
import pickle

with open('../../objects_jsons/celeba_test_set.pkl', 'rb') as f:
    test_set = pickle.load(f)

with open('../../objects_jsons/celeba_train_set.pkl', 'rb') as f:
    train_set = pickle.load(f)

test_set = list(test_set)
train_set = list(train_set)

print(test_set[:5])
print(train_set[:5])

test_set_f = [x[0] for x in test_set]
test_set_labels = [x[1] for x in test_set]

train_set_f = [x[0] for x in train_set]
train_set_labels = [x[1] for x in train_set]


[('098148.jpg', 'Noomi_Rapace'), ('036525.jpg', 'Jim_Parsons'), ('053573.jpg', 'Yasemin_Allen'), ('036583.jpg', 'Luiz_Razia'), ('076222.jpg', 'Matt_Sorum')]
[('023719.jpg', 'Mari_Abe'), ('055990.jpg', 'Derek_Mears'), ('069471.jpg', 'Noomi_Rapace'), ('122978.jpg', 'Neela_Rasgotra'), ('066968.jpg', 'Neela_Rasgotra')]


In [7]:
zoom_factor = 224/256

for i in range(len(celeba256_f)):
    print(i / len(celeba256_f))
    inpainted_f = celeba256_f[i]
    original_f = os.path.basename(inpainted_f)[:10]
    # print(original_f)

    train_ind = None
    test_ind = None
    label = None
    try:
        train_ind = train_set_f.index(original_f)
        label = train_set_labels[train_ind]
    except ValueError:
        pass

    try:
        test_ind = test_set_f.index(original_f)
        label = test_set_labels[test_ind]
    except ValueError:
        pass
        
    savename = original_f + "_masked_224.png"
    savedir = None
    if train_ind is None:
        savedir = '../../dataset_masked/val/' + label + '/'
    elif test_ind is None:
        savedir = '../../dataset_masked/train/' + label + '/'
    else:
        print("error: img not in test set or train set")
        print("img: " + original_f)
        continue

    if not os.path.isdir(savedir):
        os.mkdir(savedir)

    if os.path.isfile(os.path.join(savedir, savename)):
        continue

    img_256 = plt.imread(inpainted_f)
    if img_256.shape == (256,256,4):
        img_256 = img_256[:,:,:-1]
    try:
        assert(img_256.shape == (256, 256, 3))
    except AssertionError:
        # print(original_f)
        # print(label)
        # print(img_256.shape)

        try:
            assert(img_256.shape == (253,256,4))
            tmp = np.zeros((256,256,3))
            img_256 = img_256[:,:,:-1]
            tmp[:-3,:,:] = img_256
            tmp[-3:,:,:] = img_256[-3:,:,:]
            img_256 = tmp

        except AssertionError:
            print(original_f)
            print(label)
            print(img_256.shape)
            continue

    assert(img_256.shape == (256,256,3))

    img_224 = zoom(img_256, (zoom_factor, zoom_factor, 1))
    assert(img_224.shape[0] == 224)
    assert(img_224.shape[1] == 224)

    img_224 += np.abs(np.min([np.min(img_224), 0.]))
    img_224 /= np.max([np.max(img_224), 1.])
    # print(np.max(img_224), np.min(img_224))
    plt.imsave(os.path.join(savedir,savename), img_224)


0.0
6.654245408570669e-05
0.00013308490817141337
0.00019962736225712004
0.00026616981634282674
0.0003327122704285334
0.0003992547245142401
0.0004657971785999468
0.0005323396326856535
0.0005988820867713601
0.0006654245408570668
0.0007319669949427735
0.0007985094490284802
0.0008650519031141869
0.0009315943571998936
0.0009981368112856003
0.001064679265371307
0.0011312217194570137
000036.jpg
Frankie_J
(255, 256, 4)
0.0011977641735427202
0.0012643066276284269
0.0013308490817141336
0.0013973915357998403
0.001463933989885547
0.0015304764439712537
0.0015970188980569604
0.001663561352142667
0.0017301038062283738
0.0017966462603140805
0.0018631887143997872
0.0019297311684854936
0.0019962736225712006
0.0020628160766569073
0.002129358530742614
0.0021959009848283207
0.0022624434389140274
0.0023289858929997336
0.0023955283470854403
0.002462070801171147
0.0025286132552568537
0.0025951557093425604
0.002661698163428267
0.002728240617513974
0.0027947830715996805
0.002861325525685387
0.002927867979771094

In [ ]:
a = np.arange(25).reshape((5,5))
a

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])

In [ ]:
a[1:3:-1,:]

array([], shape=(0, 5), dtype=int64)